# Setup 

In [54]:
import google.generativeai as gai
import json, textwrap
import pandas as pd
from IPython.display import Markdown
gemini_key = json.load(open('keys.json'))['gemini']['api-key']
gai.configure(api_key=gemini_key)
model = gai.GenerativeModel('gemini-pro')

# Define helper functions

In [106]:
test = "a"
any(x in test for x in["a","b","c"])

True

In [232]:
def reformat_response(resp):
    return "\n".join([line for line in resp.text.split('\n') if line.split()[0]==line.split()[0].upper() and line!="\n" and "`" not in line])

# Start conversation

In [189]:
"test".isalpha()

True

In [257]:
instruction_format = "Use the _ command to _."

In [303]:
def get_instructions(assignment, model):
    chat = model.start_chat(history=[])
    response = chat.send_message('Generate a sql query with each command on a different line such that it satisfies the following: '+assignment+ ". Do not explain and give no other text.")
    ret = []
    for query_line in reformat_response(response).split('\n'):
        print(query_line)
        ret+= [chat.send_message("Explain what the command "+query_line+"is doing in one sentence on one line without any formatting and without revealing the command.").text.split("\n")[0]]
    return ret, chat

In [299]:
def show_instructions(instructions):
    ret = f"The query should be made of {len(instructions)} commands that {'does' if len(instructions)==1 else 'do'} the following:\n "
    for i, instruction in enumerate(instructions):
        ret+= f"{i+1}. {instruction}\n "
    return ret

In [304]:
print(show_instructions(get_instructions("get every person in the database",model)[0]))

SELECT
    *
FROM
The query should be made of 3 commands that do the following:
 1. SELECT retrieves rows from a database.
 2. * (asterisk) selects all columns in a table.
 3. FROM specifies the table to retrieve data from.
 
